# Initializing your environment

## Installation as PyMISP user

The quick and dirty way:

```bash
sudo pip3 install pymisp
```

The clean approach as user:

```bash
pip3 install --user pymisp
```

## Installation as PyMISP developer (recommended for this session)


```bash
git clone https://github.com/MISP/PyMISP.git

cd PyMISP

virtualenv -p python3 pymisp-env
source pymisp-env/bin/activate

pip install -e . 
```


# Setting up of jupyter

**We assume you're in a virtual environment**

If you want to follow along this workshop on your computer, this is the way to go:


```bash
pip3 install jupyter
cd docs/tutorial
jupyter-notebook
```


## MISPEvent

`MISPEvent` is the main class to use when you want to create/update events on a MISP instance.

In [ ]:
from pymisp import MISPEvent

event = MISPEvent()

event.info = 'This is my new MISP event'  # Required
event.distribution = 0  # Optional, defaults to MISP.default_event_distribution in MISP config
event.threat_level_id = 2  # Optional, defaults to MISP.default_event_threat_level in MISP config
event.analysis = 1  # Optional, defaults to 0 (initial analysis)

print(event.to_json())

## Tag Event

First example of helper aiming to make your life easier.

In [ ]:
event.add_tag('tlp:white')

print(event.to_json())

## Set the Event date


The date can be in many different formats. This helper makes sure it normalizes it in a way that will be understood by your MISP instance.

In [ ]:
# As text
event.set_date('2018-04-13')
print('Simple', event.date)

# Some weird text format (anything supported by dateparse will work)
event.set_date('Sat Oct 11 00:13:46 2017')
print('Messy', event.date)

# datetime.date
from datetime import date
d = date.today()
print(type(d))
event.set_date(d)
print(event.date)

# datetime.datetime => MISP expects a day, so the hour will be dropped.
from datetime import datetime
d = datetime.now()
print(type(d))
event.set_date(d)
print(event.date)

## Add Attribute to event

More useful things: adding attributes to an event.

Attributes have a bunch of parameters you can pass (if you feel like it). If you don't pass them, they'll be automatically set depending on their sane defaults.

The parameters are the following:
* **type** (required)
* **value** (required)
* **category**: [see default](https://github.com/MISP/PyMISP/blob/master/pymisp/data/describeTypes.json)
* **to_ids**: [see default](https://github.com/MISP/PyMISP/blob/master/pymisp/data/describeTypes.json)
* **distribution**: defaults to inherit from parent (event or object)
* **disable_correlation**: true for a normal attribute, fallback to the value defined in the template object if relevant
* **data**: only for malware-sample or attachment, BytesIO object of the file. If it is a malware, the sample is decrypted in memory

In [ ]:
attribute = event.add_attribute('ip-dst', '8.8.8.8')  # Minimal parameters

print(type(attribute))

print(attribute.to_json())

## Set parameters (inline)

This is the way to pass other parameters

In [ ]:
attribute_second = event.add_attribute('ip-dst', '8.8.8.9', disable_correlation=True)

print(attribute_second.to_json())

## Modify existing attribute

Every parameter can be modified in a pythonic way.

In [ ]:
attribute.to_ids = False

print(attribute.to_json())

## Tag Attribute

In [ ]:
# Using the list of attributes in the event
event.attributes[0].add_tag('tlp:green')

# ... or the variable we got from `add_attribute`
attribute_second.add_tag('tlp:amber')

print(attribute_second.to_json())

In [ ]:
print(event.to_json())

## Soft delete attribute

**Important note**: the default approach to *delete* on MISP is to do a soft delete (meaning the attribute is not displayed on the default view on MISP). The reason we do it this way is that it allows to push *delete* updates to instances we synchronize with.

The delete method will set the default parameter of the attribute to `True`.

In [ ]:
attribute.delete()
print(attribute.to_json())

## Mark event as published

Same idea: you can set the published flag from the api

In [ ]:
event.publish()
print(event.published)

## MISPAttribute

In [ ]:
attr_type = 'ip-dst'
value = '1.1.1.1'

In [ ]:
from pymisp import MISPAttribute

# Attribute data already defined
attribute = MISPAttribute()
attribute.type = attr_type
attribute.value = value
print(attribute)

In [ ]:
# An attribute can also be loaded directly from a JSON
json = '''{
    "type": "ip-dst",
    "value": "127.0.0.1",
    "category": "Network activity",
    "to_ids": false
    }'''

attribute = MISPAttribute()
attribute.from_json(json)
print(attribute)

## MISPObject

Objects in MISP are a way to group attributes together in a way that makes sense. The objects are based on templates that are bundled in the library itself.

**Note**: you can use your own templates, we will see how later

In [ ]:
from pymisp import MISPObject

circl_attr = event.add_attribute('ip-dst', '149.13.33.14')


misp_object = MISPObject('domain-ip', standalone=False, default_attributes_parameters=circl_attr)
# Notes: 
# * standalone: this object will be attached to a MISPEvent, so the references will be in the dump
# * default_attributes_parameters: keep parameters from a MISPAttribute (useful when expanding a existing one) 
misp_object.comment = 'My Fancy new object'

obj_attr = misp_object.add_attribute('domain', value='circl.lu')
obj_attr.add_tag('tlp:green')
misp_object.add_attribute('ip', value='149.13.33.14')
misp_object.add_attribute('first-seen', value='2018-04-11')
misp_object.add_attribute('last-seen', value='2018-06-11')

event.add_object(misp_object)
print(event.to_json())


## Short version to add an object to a MISPEvent

You can also add the object directly in a misp event this way

In [ ]:
from pymisp import MISPObject

misp_object = event.add_object(name='domain-ip', comment='My Fancy new object, in one line')

obj_attr = misp_object.add_attribute('domain', value='circl.lu')
obj_attr.add_tag('tlp:green')
misp_object.add_attribute('ip', value='149.13.33.14')
misp_object.add_attribute('first-seen', value='2018-04-11')
misp_object.add_attribute('last-seen', value='2018-06-11')

misp_object.add_attributes('ip', {'value': '10.8.8.8', 'to_ids': False}, '10.9.8.8')


misp_object.add_reference(obj_attr.uuid, 'related-to', 'Expanded with passive DNS entry')

print(event.to_json())


## New first/last seen

In [ ]:
from pymisp import MISPObject

misp_object = event.add_object(name='domain-ip', comment='My Fancy new object, in one line')

obj_attr = misp_object.add_attribute('domain', value='circl.lu')
obj_attr.add_tag('tlp:green')
misp_object.add_attribute('ip', value='149.13.33.14')

misp_object.first_seen = '2018-04-11'
misp_object.last_seen = '2018-06-11T23:27:40.23356+07:00'

print(misp_object.last_seen)

misp_object.add_attributes('ip', {'value': '10.8.8.8', 'to_ids': False}, '10.9.8.8')


misp_object.add_reference(obj_attr.uuid, 'related-to', 'Expanded with passive DNS entry')

print(event.to_json(indent=2))

## Dump valid MISP Event ready to push to MISP

We've been using `to_json` a lot. The thing you should know is that every python MISP objects have this method, and it **always** returns a valid json blob you can send to MISP.

In [ ]:
print(event.to_json())

# Update an existing MISPEvent

We were creating new events, but you will also want to update an existing one.

In [ ]:
from pymisp import MISPEvent

existing_event = MISPEvent()
existing_event.load_file('../../tests/mispevent_testfiles/existing_event.json')

print(existing_event.attributes[0])
print(existing_event.attributes[0].tags)
print(existing_event.attributes[0].timestamp)
print(existing_event.attributes[0].to_json())

## Edit, removes the timestamp when exporting

If you tried to edit an event manually, and never got the updates on the instance, it is probably because the timestamps weren't updated/removed. Or you removed them all, and adding a single tag was making every attributes as new.

PyMISP got you covered.

In [ ]:
existing_event.attributes[0].add_tag('tlp:white')
print(existing_event.attributes[0].to_json())

***

# Getting the API key (automatically generated on the training VM)

In [ ]:
from pathlib import Path

api_file = Path('apikey')
if api_file.exists():
    misp_url = 'http://127.0.0.1'
    misp_verifycert = False
    with open(api_file) as f:
        misp_key = f.read().strip()
    print(misp_key)
else:
    print("Unable to find the api key")

## Initialize variables if you run the notebook locally

In [ ]:
# The URL of the MISP instance to connect to
misp_url = 'https://127.0.0.1:8443/'
# Can be found in the MISP web interface under 
# http://+MISP_URL+/users/view/me -> Authkey
misp_key = 'JGKEJJqffwZd7gOo73O3F89FL8POqLXYvIY9Qevd'
# Should PyMISP verify the MISP certificate
misp_verifycert = False

In [ ]:
from pymisp import PyMISP
import urllib3
urllib3.disable_warnings()

misp = PyMISP(misp_url, misp_key, misp_verifycert)

# Interacting with a MISP instance

## Creating An Event

### Directly

In [ ]:
misp.toggle_global_pythonify()  # Returns PyMISP objects whenever possible, allows to skip pythonify

event = misp.add_event({'distribution': 1, "threat_level_id": 1, "analysis": 1, 'info':"Event from notebook"})
print("Event id: %s" % event.id)

### Using the MISPEvent constructor

In [ ]:
from pymisp import MISPEvent

event_obj = MISPEvent()
event_obj.distribution = 1
event_obj.threat_level_id = 1
event_obj.analysis = 1
event_obj.info = "Event from notebook 2"
event = misp.add_event(event_obj, pythonify=True)
event_id = event.id
print("Event id: %s" % event_id)

### Fetching an event

In [ ]:
event_id = 198

In [ ]:
# Fetch by ID
event = misp.get_event(event_id)
print(event)

### Add an attribute to an event

In [ ]:
value = "9.8.8.8"
attr_type = "ip-src"
value = "8.8.8.8"
category = "Network activity"
to_ids = False

In [ ]:
from pymisp import MISPAttribute

# Attribute data already defined
attribute = MISPAttribute()
attribute.type = attr_type
attribute.value = value
attribute.category = category
attribute.to_ids = to_ids

attribute_to_change = misp.add_attribute(event_id, attribute, pythonify=True)
print(attribute_to_change.id, attribute_to_change)

### Update existing event

In [ ]:
from pymisp import MISPAttribute, MISPObject

attr_type = "ip-src"
value = "20.8.8.8"
category = "Network activity"
to_ids = False

# Attribute data already defined
attribute = MISPAttribute()
attribute.type = attr_type
attribute.value = value
attribute.category = category
attribute.to_ids = to_ids

event = misp.get_event(event_id)

## Add the attribute to the event
event.add_attribute(**attribute)
event.add_attribute(type='domain', value='circl.lu', disable_correlation=True)

mispObject = MISPObject('file')
mispObject.add_attribute('filename', type='filename',
                         value='filename2.exe',
                         Tag=[{'name': 'tlp:white'}])

event.add_object(mispObject)

## Push the updated event to MISP
event_dict = misp.update_event(event)
print(event_dict)

### Full example

In [ ]:
from pymisp import MISPEvent, MISPObject

event = MISPEvent()
event.info = 'This is my new MISP event'  # Required
event.distribution = 0  # Optional, defaults to MISP.default_event_distribution in MISP config
event.threat_level_id = 2  # Optional, defaults to MISP.default_event_threat_level in MISP config
event.analysis = 1  # Optional, defaults to 0 (initial analysis)

mispObject = MISPObject('file')
mispObject.add_attribute('filename', type='filename',
                         value='filename.exe',
                         Tag=[{'name': 'tlp:amber'}])

event.add_object(mispObject)

print(misp)
existing_event = misp.add_event(event, pythonify=True)
print(existing_event)
mispObject = MISPObject('file')
mispObject.add_attribute('filename', type='filename',
                         value='filename2.exe',
                         Tag=[{'name': 'tlp:white'}])

existing_event.add_object(mispObject)
print(existing_event.to_json())

res = misp.update_event(existing_event)
existing_event = MISPEvent()
existing_event.load(res)
print(existing_event.to_json())

# Direct call, no validation

In [ ]:
misp.direct_call(f'attributes/add/{event_id}', {'type': 'ip-dst', 'value': '8.11.8.8'})

In [ ]:
misp.direct_call('events')

## User

In [ ]:
misp.users()

In [ ]:
misp.add_user({'email': 'bar@foo.de'})

## Organisations

In [ ]:
misp.organisations()

# Searches

# Index Search (fast, only returns events metadata)

### Search unpublished events

In [ ]:
r = misp.search(published=False, metadata=True)
print(r)

### Get the meta data of events

In [ ]:
r = misp.search(eventid=[1,2,3], metadata=True, pythonify=True)
r

### Search Tag & mix with other parameters

In [ ]:
r = misp.search(tags=['tlp:white'], metadata=True, pythonify=True)
for e in r:
    print(e)

In [ ]:
print('No attributes are in the event', r[0].attributes)

In [ ]:
r = misp.search(tags=['%tlp:amber%'], pythonify=True)
print(r[0].tags)

### Search updated events

In [ ]:
r = misp.search(timestamp='1h', metadata=True)
r

In [ ]:
r = misp.search(timestamp=['2h', '1h'])

### Search published events

In [ ]:
r = misp.search(publish_timestamp='1h', metadata=True)
r

## Getting timestamps

In [ ]:
from datetime import datetime, date, timedelta
from dateutil.parser import parse

int(datetime.now().timestamp())

d = parse('2018-03-24')
int(d.timestamp())

today = int(datetime.today().timestamp())
yesterday = int((datetime.today() - timedelta(days=1)).timestamp())

print(today, yesterday)



In [ ]:
complex_query = misp.build_complex_query(or_parameters=['uibo.lembit@mail.ee', '103.195.185.222'])
complex_query

In [ ]:
complex_query = misp.build_complex_query(or_parameters=['59.157.4.2', 'hotfixmsupload.com', '8.8.8.8'])
events = misp.search(value=complex_query, pythonify=True)

for e in events:
    print(e)

## Search for attributes

In [ ]:
r = misp.search(controller='attributes', value='wrapper.no', event_timestamp='5d')  # only consider events updated since this timestamp
print(r)

In [ ]:
# Search attributes (specified in controller) where the attribute type is 'ip-src'
# And the to_ids flag is set
attributes = misp.search(controller='attributes', type_attribute='ip-src', to_ids=0, pythonify=True)

event_ids = set()
for attr in attributes:
    event_ids.add(attr.event_id)

# Fetch all related events
for event_id in event_ids:
    event = misp.get_event(event_id)
    print(event.info)

## Output formats

In [ ]:
r = misp.search(controller='attributes', value='8.8.8.8', return_format='csv', headerless=True)
print(r)

In [ ]:
r = misp.search(controller='events', value='9.8.8.8', return_format='stix')
print(r)

## Search in logs

In [ ]:
logs = misp.search_logs(model='Tag', title='tlp:white')
print(logs)

In [ ]:
logs = misp.search_logs(model='Event', pythonify=True)
for l in logs[:20]:
    print(l.title)